# Signature exports

This workbook exemplifies how to export various types of signatures for storage in [GeMS](https://github.com/bedapub/GeMS). 

In [50]:
import besca as bc
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
from scipy import sparse, io
import os
import time
import logging
import seaborn as sns
sc.logging.print_header()

# for standard processing, set verbosity to minimum
sc.settings.verbosity = 0  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80)
version = '2.8'
start0 = time.time()

scanpy==1.6.0 anndata==0.7.5 umap==0.3.10 numpy==1.20.1 scipy==1.5.1 pandas==1.2.3 scikit-learn==0.24.1 statsmodels==0.10.2 python-igraph==0.9.0 louvain==0.7.0 leidenalg==0.8.3


In [5]:
#define standardized filepaths based on above input
root_path = os.getcwd()
bescapath_full = os.path.dirname(bc.__file__)
bescapath = os.path.split(bescapath_full)[0]

analysis_name = 'sw_besca24' ## your analysis name, as per standard workflow
species='human' ## or mouse for now
conversion=False
#analysis_name = 'standard_workflow_besca2_CLR' #use _CLR or _DSB for citeseq analysis
annot_author = 'annot_author' ### replace with own userid

# Choose the clustering to base the annotation on.
# leiden (RNA), citeseq (protein)
clusters='leiden' 
results_folder = os.path.join(root_path, 'analyzed')
results_folder = os.path.join(results_folder, analysis_name)

### Set global DE cutoffs 
myfc=1 # (log2FC)
mypval=0.05 # adjusted p-val

use_example_dataset = True ### Set to False if own data

In [ ]:

if use_example_dataset:
    analysis_name='Granja_processed'
    results_folder = os.path.split(os.getcwd())[0] + '/besca/datasets/data/'
    clusters='leiden'
    # This line will either download, or load the datasets
    adata = bc.datasets.Granja2019_processed()
    # This line exports the annotation for the annotation.

### Example 1: export cell type markers 

In [18]:
### Metadata setup for cell type signatures
User=annot_author
Source='internal scseq'  
Subtype='healthy' # or healthy, onc, ...
domain='cell marker'
studyID='Granja2019' # replace with the scMongoDB studyID
analysisID=analysis_name
genesetname=studyID+'_dblabel'
suffix='_BMPBMC_user' # replace with specific info, e.g. _mc38_pcs
signature_dict={}

In [19]:
### Perform DE cells of each celltype3 vs. all other cells
DEgenes=bc.tl.dge.get_de(adata,'dblabel',demethod='wilcoxon',topnr=5000, logfc=myfc,padj=pval)


##### Example 1: naive B cell markers


In [ ]:
### Select only top 35 genes (in order of p-val) for 2 cell types and plot expression per cell type
coi='naive B cell' # full dblabel
coishort='NaiBcell' # dblabel short
tops=list(DEgenes[coi].sort_values('Log2FC',ascending=False)['Name'][0:35])
sc.pl.dotplot(adata, var_names=tops,groupby='celltype3')


setName=coishort+suffix
desc='Genes higher expressed in '+ coi + ' vs. all other cells in Kotliarov et al. 2019 data; coefs are log2FC'
pdout=DEgenes[coi].sort_values('Log2FC',ascending=False)[0:30]
genes="\t".join(list(pdout['Name'].astype(str) + " | " + pdout['Log2FC'].round(2).astype(str)))
signature_dict[setName] = bc.tl.sig.make_gmtx(setName,desc,User,Source,Subtype,domain,genesetname,genes,studyID,analysisID)


##### Example 2: pDC markers

In [ ]:
### Select only top 15 genes (in order of p-val) for 2 cell types and plot expression per cell type
coi='plasmacytoid dendritic cell' # full dblabel
coishort='pDC' #dblabel short
tops=list(DEgenes[coi].sort_values('Log2FC',ascending=False)['Name'][0:35])
sc.pl.dotplot(adata, var_names=tops,groupby='celltype3')

setName=coishort+suffix
desc='Genes higher expressed in '+ coi + ' vs. all other cells in MC38 in vivo exp ID tumor; coefs are log2FC'
pdout=DEgenes[coi].sort_values('Log2FC',ascending=False)[0:30]
genes="\t".join(list(pdout['Name'].astype(str) + " | " + pdout['Log2FC'].round(2).astype(str)))
signature_dict[setName] = bc.tl.sig.make_gmtx(setName,desc,User,Source,Subtype,domain,genesetname,genes,studyID,analysisID)



And now export the signatures to a .gmtx file

In [ ]:
outgmtfile=results_folder+'/Celltypemarkers.gmtx' ### replace this with the path and file name of interest

bc.tl.sig.write_gmtx_forgems(signature_dict, outgmtfile)

### Example 2: export condition markers

This is a similar setup to a perturbation setting. Here, we only focus on a cell of a specific type and calculate differences PBMC vs. bone marrow, thus the example is a tissue marker.  

In [39]:
### Metadata setup for cell type signatures
User=annot_author
Source='internal scseq'  
Subtype='healthy' # or healthy, onc, ...
domain='tissue marker' ### change to perturbation if needed
studyID='Granja2019' # replace with the scMongoDB studyID
analysisID=analysis_name
genesetname=studyID+'_BMvsPBMC_higher' # for perturbation e.g. Pembro_induced_MC38CD8Tcell
suffix='_'+studyID # replace with specific info, e.g. _mc38_pcs
signature_dict={}


In [43]:
###### Genes DE expressed BM vs. PBMC in CD8+ T cells

In [35]:
totest=adata[adata.obs['Type'].isin(['BMMC','PBMC'])].copy()
totest=totest[totest.obs['celltype2']=='CD8-positive, alpha-beta T cell']

In [36]:

DEgenes=bc.tl.dge.get_de(totest,'Type',demethod='wilcoxon',topnr=5000, logfc=myfc,padj=mypval) ### Would be treatment_id

Trying to set attribute `.uns` of view, copying.


In [40]:
DEgenes['BMMC'].sort_values('Log2FC', ascending=False)

,Name,Score,Log2FC,P.adj
275,MTRNR2L1,3.382508,7.325880,1.658312e-02
42,SCGB3A1,9.673761,5.456781,6.321058e-20
35,HBB,10.380047,5.323501,6.049603e-23
89,HBA2,6.697232,5.301816,1.652583e-09
172,HBA1,4.556225,4.140492,2.008633e-04
...,...,...,...,...
134,CBX4,5.460600,1.036407,2.466074e-06
27,MGAT4A,11.940870,1.035104,1.950649e-30
133,SYAP1,5.482232,1.032961,2.191131e-06
252,GPR155,3.545514,1.030180,9.821738e-03


In [46]:
setName='BMMChigh_inCD8Tcell'+suffix # for perturbation e.g. Pembro_induced_MC38CD8Tcell_T1
desc='Genes higher expressed in bone marrow vs. PBMC in CD8-positive T cells in Granja et al. dataset; coefs are log2FC'
# for perturbation e.g. 'Genes higher expressed in Pembro vs. vehicle in CD8-positive T cells in MC38 in vivo exp. ID time T2; coefs are log2FC'
pdout=DEgenes['BMMC'].sort_values('Log2FC', ascending=False)
genes="\t".join(list(pdout['Name'].astype(str) + " | " + pdout['Log2FC'].round(2).astype(str)))
signature_dict[setName]=bc.tl.sig.make_gmtx(setName,desc,User,Source,Subtype,domain,genesetname,genes,studyID,analysisID)

Metadata for signature BMMChigh_inCD8Tcell_Granja2019 successfully captured


In [48]:
setName='PBMChigh_inCD8Tcell'+suffix # for perturbation e.g. Pembro_induced_MC38CD8Tcell_T1
desc='Genes higher expressed in PBMC vs. bone marrow in CD8-positive T cells in Granja et al. dataset; coefs are log2FC'
# for perturbation e.g. 'Genes higher expressed in Pembro vs. vehicle in CD8-positive T cells in MC38 in vivo exp. ID time T2; coefs are log2FC'
pdout=DEgenes['PBMC'].sort_values('Log2FC', ascending=False)
genes="\t".join(list(pdout['Name'].astype(str) + " | " + pdout['Log2FC'].round(2).astype(str)))
signature_dict[setName]=bc.tl.sig.make_gmtx(setName,desc,User,Source,Subtype,domain,genesetname,genes,studyID,analysisID)

Metadata for signature PBMChigh_inCD8Tcell_Granja2019 successfully captured


In [51]:
signature_dict

{'BMMChigh_inCD8Tcell_Granja2019': {'setName': 'BMMChigh_inCD8Tcell_Granja2019',
  'desc': 'Genes higher expressed in bone marrow vs. PBMC in CD8-positive T cells in Granja et al. dataset; coefs are log2FC',
  'User': 'annot_author',
  'Source': 'internal scseq',
  'Subtype': 'healthy',
  'geneset': 'Granja2019_BMvsPBMC_higher',
  'domain': 'tissue marker',
  'studyID': 'Granja2019',
  'analysisID': 'Granja_processed',
  'genes|logFC': 'MTRNR2L1 | 7.33\tSCGB3A1 | 5.46\tHBB | 5.32\tHBA2 | 5.3\tHBA1 | 4.14\tGSTM4 | 3.54\tMETRNL | 2.83\tRGS1 | 2.82\tNR4A2 | 2.72\tCOL18A1 | 2.68\tENC1 | 2.46\tZNF331 | 2.35\tDUSP5 | 2.35\tCCL3 | 2.26\tPOMC | 2.11\tODC1 | 2.1\tPTP4A1 | 2.07\tRBM38 | 1.92\tHMGB2 | 1.88\tGADD45A | 1.78\tGSTM3 | 1.68\tIRS2 | 1.66\tSNN | 1.59\tSOCS1 | 1.58\tGNAQ | 1.58\tHLA-DQB1 | 1.52\tXCL2 | 1.52\tMTFP1 | 1.5\tSYTL3 | 1.48\tTNFAIP3 | 1.48\tCREM | 1.48\tCHPT1 | 1.43\tRNF125 | 1.42\tPIK3R1 | 1.42\tCD45RA_PROT | 1.4\tP2RY10 | 1.39\tPDE4B | 1.36\tCSRNP1 | 1.36\tOSER1 | 1.34\tCXCR4

And now export the signatures to a .gmtx file


In [79]:

outgmtfile=results_folder+'/Tissuemarkers.gmtx' ### replace this with the path and file name of interest

bc.tl.sig.write_gmtx_forgems(signature_dict, outgmtfile)

In [49]:
! jupyter nbconvert --to html Signature_exports.ipynb


[NbConvertApp] Converting notebook Signature_exports.ipynb to html
[NbConvertApp] Writing 1307869 bytes to Signature_exports.html
